In [1]:
%pwd 

'd:\\pythonProjects\\MLflow_ML_project\\research'

In [2]:
import os 

os.chdir('../')
%pwd  

'd:\\pythonProjects\\MLflow_ML_project'

In [3]:
import pandas as pd

data=pd.read_csv('artifacts/data_ingestion/wine-quality/WineQT.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [25]:
data.dtypes.to_dict()

{'fixed acidity': dtype('float64'),
 'volatile acidity': dtype('float64'),
 'citric acid': dtype('float64'),
 'residual sugar': dtype('float64'),
 'chlorides': dtype('float64'),
 'free sulfur dioxide': dtype('float64'),
 'total sulfur dioxide': dtype('float64'),
 'density': dtype('float64'),
 'pH': dtype('float64'),
 'sulphates': dtype('float64'),
 'alcohol': dtype('float64'),
 'quality': dtype('int64'),
 'Id': dtype('int64')}

In [5]:
# EDA 
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Id                      0
dtype: int64

In [6]:
data.shape 

(1143, 13)

In [7]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'Id'],
      dtype='object')

In [8]:
# entity 
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str 
    unzip_data_dir: Path 
    all_schema: dict 

In [9]:
# config
from mlProject.constants import * 
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            param_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH,
            ):
        self.config=read_yaml(config_filepath)
        self.param=read_yaml(param_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) 

    def get_data_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            unzip_data_dir=config.unzip_data_dir,
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema 
        )
        return data_validation_config

In [37]:
# component
import os 
from mlProject import logger

class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def validate_all_columns(self)->bool:
        try:
            validate_status=None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=data.dtypes.to_dict()
            all_schema=self.config.all_schema
            
            for (key1,value1), (key2,value2) in zip(all_cols.items(), all_schema.items()):
                if key1 != key2 or value1 !=value2:         
                    validate_status=False 
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f'Validation status: {validate_status}')
                else:
                    validate_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f'Validation status: {validate_status}')
            return validate_status
        except Exception as e:
            raise e 

In [40]:
# pipeline 
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e 

[2025-01-04 12:47:16,564: INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-01-04 12:47:16,565: INFO:common: yaml file: params.yaml loaded successfully]
[2025-01-04 12:47:16,568: INFO:common: yaml file: schema.yaml loaded successfully]
[2025-01-04 12:47:16,569: INFO:common: created directory at: artifacts]
[2025-01-04 12:47:16,570: INFO:common: created directory at: artifacts/data_validation]
